# CV RAG Training Notebook

Tento notebook umožňuje ruční spouštění a testování trénovacího procesu RAG systému pro vyhledávání v životopisech.

## Co se děje během trénování:

1. **Načtení CV** - Načtou se všechny DOCX soubory z datového adresáře
2. **Setup embeddings** - Připraví se Azure OpenAI embeddings
3. **Setup vector store** - Vytvoří se prázdný ChromaDB vectorstore
4. **Inicializace retrieveru** - ParentDocumentRetriever:
   - Rozdělí každé CV na parent chunks (~2000 znaků)
   - Rozdělí parent chunks na child chunks (~400 znaků)
   - Child chunks se indexují do vectorstore (pro vyhledávání)
   - Parent chunks se ukládají do docstore (pro context)
5. **Test retrieval** - Vyzkouší několik testovacích dotazů

## Import knihoven a konfigurace

In [1]:
import sys
import logging
from pathlib import Path


# Přidat parent directory do sys.path pro importy
sys.path.insert(0, str(Path.cwd().parent))

# Nastavení logging pro viditelnost procesu
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import aplikačních modulů
from src.config import AppConfig
from src.document_loader import CVDocumentLoader
from src.embeddings import EmbeddingsManager
from src.vector_store import VectorStoreManager
from src.parent_retriever import CVParentRetriever

print("✓ Knihovny načteny")

✓ Knihovny načteny


## Načtení konfigurace

In [2]:
# Načtení konfigurace z .env souboru
config = AppConfig()

print("\n📋 Konfigurace:")
print(f"  Data directory: {config.rag.data_directory}")
print(f"  Vector store: {config.rag.persist_directory}")
print(f"  Parent chunk size: {config.rag.parent_chunk_size}")
print(f"  Child chunk size: {config.rag.child_chunk_size}")
print(f"  Batch size: {config.azure.batch_size}")
print(f"  Batch delay: {config.azure.batch_delay}s")


📋 Konfigurace:
  Data directory: ./data/OneDrive_2025-12-16
  Vector store: ./chroma_db
  Parent chunk size: 2000
  Child chunk size: 400
  Batch size: 5
  Batch delay: 2.0s


## KROK 1: Načtení CV dokumentů

Načte všechny `.docx` soubory z datového adresáře a převede je na `Candidate` objekty.

In [3]:
print("\n" + "="*80)
print("KROK 1: Načítání CV dokumentů")
print("="*80)

# Vytvoření loaderu
loader = CVDocumentLoader(config.rag.data_directory_ntb)

# Načtení všech CV
candidates = loader.load_all_cvs()

print(f"\n✓ Načteno {len(candidates)} CV")
print("\nPrvních 5 kandidátů:")
for i, candidate in enumerate(candidates[:5], 1):
    print(f"  {i}. {candidate.name} ({len(candidate.full_cv_text)} znaků)")

if len(candidates) > 5:
    print(f"  ... a dalších {len(candidates) - 5}")

2025-12-17 16:40:32,709 - src.document_loader - INFO - Found 27 DOCX files in ..\data\OneDrive_2025-12-16
2025-12-17 16:40:32,736 - src.document_loader - INFO - Loaded CV for Baláček Daniel (3020 characters)
2025-12-17 16:40:32,760 - src.document_loader - INFO - Loaded CV for Bobůrka Vojtěch (2458 characters)
2025-12-17 16:40:32,791 - src.document_loader - INFO - Loaded CV for Bronec Ondřej (3757 characters)
2025-12-17 16:40:32,815 - src.document_loader - INFO - Loaded CV for Bukovský Petr (2628 characters)
2025-12-17 16:40:32,830 - src.document_loader - INFO - Loaded CV for Bímová Kamila (2042 characters)
2025-12-17 16:40:32,850 - src.document_loader - INFO - Loaded CV for Dlugošová Lenka (2383 characters)
2025-12-17 16:40:32,868 - src.document_loader - INFO - Loaded CV for Duleba Peter (2873 characters)



KROK 1: Načítání CV dokumentů


2025-12-17 16:40:32,883 - src.document_loader - INFO - Loaded CV for Fejfarová Julia (1445 characters)
2025-12-17 16:40:32,896 - src.document_loader - INFO - Loaded CV for Fejfar Ondřej (1289 characters)
2025-12-17 16:40:32,919 - src.document_loader - INFO - Loaded CV for Gleb Tcypin (2543 characters)
2025-12-17 16:40:32,949 - src.document_loader - INFO - Loaded CV for Hlavatá Michaela (5445 characters)
2025-12-17 16:40:32,959 - src.document_loader - INFO - Loaded CV for Hlinková Zuzana (2615 characters)
2025-12-17 16:40:32,983 - src.document_loader - INFO - Loaded CV for Holman Martin (1559 characters)
2025-12-17 16:40:33,013 - src.document_loader - INFO - Loaded CV for Hrdý Daniel (2399 characters)
2025-12-17 16:40:33,048 - src.document_loader - INFO - Loaded CV for Huňa Tomáš (3930 characters)
2025-12-17 16:40:33,067 - src.document_loader - INFO - Loaded CV for Hušek Michal (2354 characters)
2025-12-17 16:40:33,089 - src.document_loader - INFO - Loaded CV for Karlovský Lukáš (3653 c


✓ Načteno 27 CV

Prvních 5 kandidátů:
  1. Baláček Daniel (3020 znaků)
  2. Bobůrka Vojtěch (2458 znaků)
  3. Bronec Ondřej (3757 znaků)
  4. Bukovský Petr (2628 znaků)
  5. Bímová Kamila (2042 znaků)
  ... a dalších 22


### Podívejte se na jedno CV

In [4]:
# Zobrazení prvních 500 znaků z prvního CV
if candidates:
    first_candidate = candidates[0]
    print(f"\n📄 Příklad CV: {first_candidate.name}")
    print(f"Délka: {len(first_candidate.full_cv_text)} znaků")
    print(f"\nPrvních 500 znaků:")
    print("-" * 80)
    print(first_candidate.full_cv_text[:500])
    print("-" * 80)


📄 Příklad CV: Baláček Daniel
Délka: 3020 znaků

Prvních 500 znaků:
--------------------------------------------------------------------------------
www.dolphinconsulting.cz	

Daniel Baláček

BI consultant 

 Praha



Key qualifications

Dashboard and report development and design

User requirements analysis and documentation

Business and data analysis



Skills & knowledge

Business intelligence

General Skills

Dashboard and report development and design 

Datawarehouse architecture principles and principles of BI

Data modeling

Business intelligence

Applications

MS Excel - advanced

Qlik Sense – advanced 

Qlik NPrinting – advanced

Q
--------------------------------------------------------------------------------


### Převod na LangChain Documents

In [5]:
# Převod kandidátů na LangChain Documents
documents = loader.convert_to_langchain_documents(candidates)

print(f"\n✓ Vytvořeno {len(documents)} LangChain Documents")
print("\nPříklad metadat prvního dokumentu:")
if documents:
    print(documents[0].metadata)

2025-12-17 16:40:33,501 - src.document_loader - INFO - Converted 27 candidates to LangChain documents



✓ Vytvořeno 27 LangChain Documents

Příklad metadat prvního dokumentu:
{'candidate_name': 'Baláček Daniel', 'source': '..\\data\\OneDrive_2025-12-16\\Baláček_Daniel_CV_EN.docx', 'type': 'cv_parent', 'filename': 'Baláček_Daniel_CV_EN.docx', 'file_size': 496940, 'text_length': 3020}


In [6]:
print(len(documents))

27


## KROK 2: Setup Azure OpenAI Embeddings

Připraví embeddings model pro vytváření vektorových reprezentací textu.

In [7]:
print("\n" + "="*80)
print("KROK 2: Setup Embeddings")
print("="*80)

embeddings_mgr = EmbeddingsManager(config.azure)

print(f"\n✓ Embeddings manager vytvořen")
print(f"  Model: {config.azure.embedding_deployment}")
print(f"  Endpoint: {config.azure.endpoint}")


KROK 2: Setup Embeddings

✓ Embeddings manager vytvořen
  Model: text-embedding-ada-002-dolphin-1
  Endpoint: https://oai-dolphin-1.openai.azure.com


### Test připojení k Azure OpenAI

In [8]:
# Test připojení
if embeddings_mgr.test_connection():
    print("\n✓ Připojení k Azure OpenAI úspěšné")
else:
    print("\n✗ Připojení k Azure OpenAI selhalo")

2025-12-17 16:40:33,553 - src.embeddings - INFO - Initializing Azure OpenAI Embeddings with deployment: text-embedding-ada-002-dolphin-1
2025-12-17 16:40:37,515 - src.embeddings - INFO - Embeddings initialized successfully
2025-12-17 16:40:45,654 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:40:45,670 - src.embeddings - INFO - Embeddings connection test successful



✓ Připojení k Azure OpenAI úspěšné


## KROK 3: Setup Vector Store

Vytvoří prázdný ChromaDB vectorstore (nebo načte existující).

In [9]:
print("\n" + "="*80)
print("KROK 3: Setup Vector Store")
print("="*80)

vs_manager = VectorStoreManager(
    config.rag,
    embeddings_mgr.get_embeddings()
)

# Smazání existujícího vectorstore (pro čisté trénování)
print("\nMažu existující vectorstore...")
vs_manager.clear_vectorstore()

# Vytvoření prázdného vectorstore
print("Vytvářím prázdný vectorstore...")
vectorstore = vs_manager.create_or_load_vectorstore()

print("\n✓ Vector store připraven")

2025-12-17 16:40:45,707 - src.vector_store - INFO - Clearing vector store at chroma_db



KROK 3: Setup Vector Store

Mažu existující vectorstore...


2025-12-17 16:40:45,898 - src.vector_store - INFO - Vector store cleared
2025-12-17 16:40:45,901 - src.vector_store - INFO - Creating new empty vector store at chroma_db


Vytvářím prázdný vectorstore...


C:\Users\jan.petr\OneDrive - dolphinconsulting.cz\Projects\rag-training\app_cvs\src\vector_store.py:66: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  self._vectorstore = Chroma(
2025-12-17 16:40:54,215 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-12-17 16:40:55,415 - src.vector_store - INFO - Empty vector store created



✓ Vector store připraven


## KROK 4: Inicializace Parent Document Retrieveru

**Toto je klíčový krok!**

ParentDocumentRetriever:
1. Rozdělí každé CV na **parent chunks** (velké kusy ~2000 znaků)
2. Rozdělí každý parent chunk na **child chunks** (malé kusy ~400 znaků)
3. **Child chunks** se indexují do vectorstore (používají se pro vyhledávání)
4. **Parent chunks** se ukládají do docstore (vracejí se jako kontext)
5. Pamatuje si mapování: který child chunk patří ke kterému parent chunku

**Batch processing:**
- Počítá skutečný počet child chunks (ne jen počet CV)
- Po každých ~50 chuncích udělá pauzu (rate limit protection)

In [10]:
print("\n" + "="*80)
print("KROK 4: Inicializace Parent Document Retriever")
print("="*80)

retriever = CVParentRetriever(
    config=config.rag,
    vectorstore=vectorstore,
    azure_config=config.azure
)

print(f"\nZpracovávám {len(documents)} dokumentů...")
print(f"Batch size: {config.azure.batch_size} chunks")
print(f"Batch delay: {config.azure.batch_delay}s\n")

# POZOR: Toto může trvat několik minut!
# Vytváří embeddingy pro všechny child chunks
retriever.initialize_retriever(documents)

print("\n✓ Retriever inicializován")

# Zkontroluj statistiky
stats = retriever.get_stats()
print(f"\nStatistiky:")
print(f"  - Parent chunks: {stats['parent_chunks']}")
print(f"  - Child chunks: {stats['child_chunks']}")

2025-12-17 16:40:55,448 - src.parent_retriever - INFO - Initializing Parent Retriever - Parent chunks: 2000, Child chunks: 400
2025-12-17 16:40:55,450 - src.parent_retriever - INFO - Docstore path: chroma_db\docstore
2025-12-17 16:40:55,453 - src.parent_retriever - INFO - Initializing retriever with 27 CV documents
2025-12-17 16:40:55,456 - src.parent_retriever - INFO - Using batch processing for retriever initialization: batch_size=5
2025-12-17 16:40:55,457 - src.parent_retriever - INFO - Pre-splitting 27 documents into child chunks...
2025-12-17 16:40:55,471 - src.parent_retriever - INFO - Total child chunks: 273
2025-12-17 16:40:55,472 - src.parent_retriever - INFO - Processing in 55 batches of ~5 chunks each
2025-12-17 16:40:55,472 - src.parent_retriever - INFO - Document 'Baláček Daniel': 10 child chunks



KROK 4: Inicializace Parent Document Retriever

Zpracovávám 27 dokumentů...
Batch size: 5 chunks
Batch delay: 2.0s



2025-12-17 16:41:03,420 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:41:03,946 - src.parent_retriever - INFO - Processed 10/273 chunks (1 batches)
2025-12-17 16:41:03,948 - src.parent_retriever - INFO - Waiting 2.0s before next batch...
2025-12-17 16:41:05,951 - src.parent_retriever - INFO - Document 'Bobůrka Vojtěch': 7 child chunks
2025-12-17 16:41:06,049 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:41:06,152 - src.parent_retriever - INFO - Processed 17/273 chunks (2 batches)
2025-12-17 16:41:06,154 - src.parent_retriever - INFO - Waiting 2.0s before next batch...
2025-12-17 16:41:08,158 - src.parent_retriever - INFO - Document 'Bronec Ondřej': 11 child chunks
2025-12-17 16:41:08,288 - httpx -


✓ Retriever inicializován

Statistiky:
  - Parent chunks: 61
  - Child chunks: 298


In [ ]:
print("\n" + "="*80)
print("KONTROLA: Docstore na disku")
print("="*80)

import os
from pathlib import Path

docstore_path = Path(config.rag.persist_directory) / "docstore"

print(f"\nDocstore cesta: {docstore_path}")
print(f"Existuje: {docstore_path.exists()}")

if docstore_path.exists():
    files = list(docstore_path.iterdir())
    print(f"Počet souborů: {len(files)}")
    
    if len(files) > 0:
        print(f"\nPrvních 10 souborů:")
        for i, file in enumerate(files[:10], 1):
            size_kb = file.stat().st_size / 1024
            print(f"  {i}. {file.name} ({size_kb:.2f} KB)")
        
        # Ukázka obsahu prvního souboru
        if files:
            first_file = files[0]
            print(f"\nObsah prvního souboru ({first_file.name}):")
            content = first_file.read_text(encoding='utf-8')
            print(content[:500] + "...")
    else:
        print("\n⚠️ Docstore složka existuje, ale je PRÁZDNÁ!")
        print("   Trénování možná selhalo nebo ještě neproběhlo.")
else:
    print("\n❌ Docstore složka NEEXISTUJE!")
    print("   Spusť initialize_retriever() pro vytvoření.")

# Úplná cesta pro průzkumník
abs_path = docstore_path.absolute()
print(f"\nÚplná cesta pro průzkumník Windows:")
print(f"{abs_path}")

In [11]:
stats

{'status': 'initialized',
 'parent_chunks': 61,
 'child_chunks': 298,
 'parent_chunk_size': 2000,
 'child_chunk_size': 400}

### Statistiky po trénování

In [12]:
# Zobrazení statistik
stats = retriever.get_stats()

print("\n📊 Statistiky:")
print(f"  Parent chunks: {stats['parent_chunks']}")
print(f"  Child chunks: {stats['child_chunks']}")
print(f"  Parent chunk size: {stats['parent_chunk_size']} znaků")
print(f"  Child chunk size: {stats['child_chunk_size']} znaků")
print(f"\n  Průměr child chunks na CV: {stats['child_chunks'] / len(documents):.1f}")
print(f"  Průměr parent chunks na CV: {stats['parent_chunks'] / len(documents):.1f}")


📊 Statistiky:
  Parent chunks: 61
  Child chunks: 298
  Parent chunk size: 2000 znaků
  Child chunk size: 400 znaků

  Průměr child chunks na CV: 11.0
  Průměr parent chunks na CV: 2.3


## KROK 5: Test Retrieval

Nyní můžete testovat vyhledávání v natrénovaném vectorstore.

In [13]:
print("\n" + "="*80)
print("KROK 5: Test Retrieval")
print("="*80)

# Testovací dotazy
test_queries = [
    "candidates with Python skills",
    "who has AWS experience",
    "Java developers"
]

for query in test_queries:
    print(f"\n🔍 Dotaz: '{query}'")
    print("-" * 80)
    
    results = retriever.retrieve(query, top_k=3)
    
    print(f"Nalezeno {len(results)} výsledků:\n")
    
    for i, doc in enumerate(results, 1):
        candidate_name = doc.metadata.get("candidate_name", "Unknown")
        content_preview = doc.page_content[:200].replace("\n", " ")
        
        print(f"{i}. {candidate_name}")
        print(f"   Délka: {len(doc.page_content)} znaků")
        print(f"   Preview: {content_preview}...")
        print()

2025-12-17 16:42:38,365 - src.parent_retriever - INFO - Retrieving documents for query: 'candidates with Python skills' (top 3)
2025-12-17 16:42:38,436 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:38,499 - src.parent_retriever - INFO - Retrieved 2 parent documents
2025-12-17 16:42:38,500 - src.parent_retriever - INFO - Retrieving documents for query: 'who has AWS experience' (top 3)



KROK 5: Test Retrieval

🔍 Dotaz: 'candidates with Python skills'
--------------------------------------------------------------------------------
Nalezeno 2 výsledků:

1. Bronec Ondřej
   Délka: 1953 znaků
   Preview: www.dolphinconsulting.cz	                Ing. Ondřej Bronec  BI consultant    Prague    Key qualifications  Data mining and data science – data cleaning, analysis, and modeling using Python and R or s...

2. Hrdý Daniel
   Délka: 1937 znaků
   Preview: www.dolphinconsulting.cz	                Daniel Hrdý  BI consultant   Prague, Czech Republic    Key qualifications  SQL procedures and queries, data modeling.  Data integration and transformation.  Us...


🔍 Dotaz: 'who has AWS experience'
--------------------------------------------------------------------------------


2025-12-17 16:42:38,645 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:38,662 - src.parent_retriever - INFO - Retrieved 2 parent documents
2025-12-17 16:42:38,664 - src.parent_retriever - INFO - Retrieving documents for query: 'Java developers' (top 3)
2025-12-17 16:42:38,746 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:38,766 - src.parent_retriever - INFO - Retrieved 3 parent documents


Nalezeno 2 výsledků:

1. Huňa Tomáš
   Délka: 1997 znaků
   Preview: www.dolphinconsulting.cz	                Tomáš Huňa  DWH & Business Intelligence Consultant  Prague  Main Qualifications  Data modeling  Data integration  Cloud Solutions Design and Management  BI App...

2. Bronec Ondřej
   Délka: 1953 znaků
   Preview: www.dolphinconsulting.cz	                Ing. Ondřej Bronec  BI consultant    Prague    Key qualifications  Data mining and data science – data cleaning, analysis, and modeling using Python and R or s...


🔍 Dotaz: 'Java developers'
--------------------------------------------------------------------------------
Nalezeno 3 výsledků:

1. Hušek Michal
   Délka: 1996 znaků
   Preview: www.dolphinconsulting.cz	    Michal Hušek  BI Consultant & developer          Key qualifications  Full stack BI solutions development  .NET development      Skills & knowledge  Programming languages  ...

2. Bukovský Petr
   Délka: 1927 znaků
   Preview: www.dolphinconsulting.cz	  Petr Buko

## Vlastní testy

Nyní můžete zadávat vlastní dotazy a testovat retrieval.

In [14]:
# Vlastní dotaz
custom_query = "frontend developer with React"  # Změňte podle potřeby

print(f"🔍 Vlastní dotaz: '{custom_query}'")
print("=" * 80)

results = retriever.retrieve(custom_query, top_k=5)

for i, doc in enumerate(results, 1):
    candidate_name = doc.metadata.get("candidate_name", "Unknown")
    print(f"\n{i}. {candidate_name}")
    print("-" * 80)
    print(doc.page_content[:5000])  # První 500 znaků
    print("...")

2025-12-17 16:42:38,788 - src.parent_retriever - INFO - Retrieving documents for query: 'frontend developer with React' (top 5)
2025-12-17 16:42:38,870 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


🔍 Vlastní dotaz: 'frontend developer with React'


2025-12-17 16:42:38,898 - src.parent_retriever - INFO - Retrieved 3 parent documents



1. Bukovský Petr
--------------------------------------------------------------------------------
www.dolphinconsulting.cz	

Petr Bukovský

DWH/BI Consultant 

 Brno



Key qualifications

Development ETL, data engineer

By education and previous experiences mechanical engineer specialized in FEA analysis and piping stress analyst in Oil and Gas industry



Skills & knowledge

Business intelligence

General

Data modeling

Business intelligence

Applications

Tabidoo (including automation using JS)

Microsoft Power BI and Power Query (basic knowledge)

MS Excel (including VBA)

ETL

Data Factory, Python, Databricks

Databases

Microsoft SQL, MS Access

Others

Basic knowledge of HTML, CSS and network setup

Python with packages for data analysis (Pandas, Numpy, Matplotlib, Seaborn)

Javascript (scripting only)

​SEO optimization

Git

Languages

Czech (native),

English (B1/B2)

German (A2)






Selected project experience

Period

Customer

Activities

2025

2024

Skoda auto

Data e

## s měřením relevance retrievalu

In [15]:
print("\n" + "="*80)
print("TEST 1: Detekce relevance - Relevantní dotaz (Python developer)")
print("="*80)

relevant_query = "Python developer with Django experience"
print(f"\nDotaz: \"{relevant_query}\"\n")

# Test s scores
print("1a. Retrieve s scores (všechny výsledky):")
print("-" * 60)
results_with_scores = retriever.retrieve_with_scores(relevant_query, top_k=5)

print(f"\n{'#':<3} | {'Kandidát':<30} | {'Score':<10} | Relevance")
print("-" * 80)

for i, result in enumerate(results_with_scores, 1):
    score = result.score
    name = result.candidate_name
    
    # Určení relevance podle score (distance metric - nižší = lepší)
    if score < 0.5:
        relevance = "VYSOKÁ ✓✓✓"
    elif score < 1.0:
        relevance = "STŘEDNÍ ✓✓"
    elif score < 1.5:
        relevance = "NÍZKÁ ✓"
    else:
        relevance = "IRELEVANTNÍ ✗"
    
    print(f"{i:<3} | {name:<30} | {score:<10.4f} | {relevance}")

# Test s filtrováním
print("\n" + "-" * 80)
print("1b. Retrieve relevant (s filtrem threshold=1.5):")
print("-" * 60)

relevant_docs = retriever.retrieve_relevant(relevant_query, top_k=5)
print(f"\nNalezeno {len(relevant_docs)} relevantních dokumentů:")
for i, doc in enumerate(relevant_docs, 1):
    print(f"  {i}. {doc.metadata.get('candidate_name', 'Unknown')}")


print("\n\n" + "="*80)
print("TEST 2: Detekce relevance - Irelevantní dotaz (React frontend)")
print("="*80)

irrelevant_query = "React frontend developer with GraphQL and TypeScript"
print(f"\nDotaz: \"{irrelevant_query}\"")
print("(Předpokládáme, že nemáme žádného React vývojáře)\n")

# Test s scores
print("2a. Retrieve s scores (všechny výsledky):")
print("-" * 60)
results_irrel = retriever.retrieve_with_scores(irrelevant_query, top_k=5)

print(f"\n{'#':<3} | {'Kandidát':<30} | {'Score':<10} | Relevance")
print("-" * 80)

for i, result in enumerate(results_irrel, 1):
    score = result.score
    name = result.candidate_name
    
    if score < 0.5:
        relevance = "VYSOKÁ ✓✓✓"
    elif score < 1.0:
        relevance = "STŘEDNÍ ✓✓"
    elif score < 1.5:
        relevance = "NÍZKÁ ✓"
    else:
        relevance = "IRELEVANTNÍ ✗"
    
    print(f"{i:<3} | {name:<30} | {score:<10.4f} | {relevance}")

# Test s filtrováním
print("\n" + "-" * 60)
print("2b. Retrieve relevant (s filtrem threshold=1.5):")
print("-" * 60)

relevant_docs_irrel = retriever.retrieve_relevant(irrelevant_query, top_k=5)
print(f"\nNalezeno {len(relevant_docs_irrel)} relevantních dokumentů")

if len(relevant_docs_irrel) == 0:
    print("\n⚠️ ŽÁDNÉ relevantní výsledky!")
    print("   → Správné chování: Nevrátit náhodné kandidáty")
else:
    for i, doc in enumerate(relevant_docs_irrel, 1):
        print(f"  {i}. {doc.metadata.get('candidate_name', 'Unknown')}")


print("\n\n" + "="*80)
print("TEST 3: Porovnání české vs anglické query")
print("="*80)

queries = [
    ("Python developer", "anglicky"),
    ("Python vývojář", "česky"),
]

for query, language in queries:
    print(f"\n{'='*60}")
    print(f"Query ({language}): \"{query}\"")
    print(f"{'='*60}")
    
    results = retriever.retrieve_with_scores(query, top_k=3)
    
    print(f"\nTop 3 výsledky:")
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result.candidate_name:<30} (score: {result.score:.4f})")

print("\n💡 TIP: Anglické dotazy obvykle dávají lepší scores")
print("   (nižší distance = vyšší podobnost)")


print("\n\n" + "="*80)
print("TEST 4: RAG Chain s relevance filtrem")
print("="*80)

# Import RAG chain
from src.rag_chain import CVRAGChain

rag_chain = CVRAGChain(config.azure, retriever)

# Test 1: Relevantní dotaz
print("\n4a. Relevantní dotaz s filtrem:")
print("-" * 60)
response1 = rag_chain.invoke("Python developer", use_relevance_filter=True)
print(f"\nOtázka: {response1.query}")
print(f"Počet kontextů: {response1.metadata['num_contexts']}")
print(f"Odpověď:\n{response1.answer}")

# Test 2: Irelevantní dotaz
print("\n\n4b. Irelevantní dotaz s filtrem:")
print("-" * 60)
response2 = rag_chain.invoke("React frontend developer", use_relevance_filter=True)
print(f"\nOtázka: {response2.query}")
print(f"Počet kontextů: {response2.metadata['num_contexts']}")
print(f"No relevant results: {response2.metadata.get('no_relevant_results', False)}")
print(f"Odpověď:\n{response2.answer}")

# Test 3: Irelevantní dotaz BEZ filtru (pro porovnání)
print("\n\n4c. Irelevantní dotaz BEZ filtru (pro porovnání):")
print("-" * 60)
response3 = rag_chain.invoke("React frontend developer", use_relevance_filter=False)
print(f"\nOtázka: {response3.query}")
print(f"Počet kontextů: {response3.metadata['num_contexts']}")
print(f"Odpověď:\n{response3.answer}")


print("\n\n" + "="*80)
print("SHRNUTÍ")
print("="*80)
print("""
1. ✓ Config: similarity_threshold = 1.5 (konfigurovatelné)
2. ✓ Nová metoda: retriever.retrieve_relevant() 
   - Filtruje podle threshold
   - Vrací jen relevantní výsledky (může být prázdné!)
3. ✓ Vylepšený RAG chain: 
   - use_relevance_filter=True (default)
   - Detekuje prázdné výsledky
   - Neposílá irelevantní kontext do LLM
4. ✓ Lepší UX: "No candidates found" místo halucinací

TIP: Nastav threshold v config.py podle potřeby:
  - < 1.0  = velmi přísné (jen velmi podobné)
  - 1.0-1.5 = standardní (dobré výsledky)
  - > 1.5  = volné (více výsledků, nižší kvalita)
""")


TEST 1: Detekce relevance - Relevantní dotaz (Python developer)

Dotaz: "Python developer with Django experience"

1a. Retrieve s scores (všechny výsledky):
------------------------------------------------------------


2025-12-17 16:42:39,058 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:39,073 - src.parent_retriever - INFO - Retrieved 5 results with scores
2025-12-17 16:42:39,080 - src.parent_retriever - INFO - Retrieving relevant documents for query: 'Python developer with Django experience' (top 5, threshold 1.5)



#   | Kandidát                       | Score      | Relevance
--------------------------------------------------------------------------------
1   | Baláček Daniel                 | 0.2978     | VYSOKÁ ✓✓✓
2   | Huňa Tomáš                     | 0.3186     | VYSOKÁ ✓✓✓
3   | Bukovský Petr                  | 0.3240     | VYSOKÁ ✓✓✓
4   | Petr Jan                       | 0.3396     | VYSOKÁ ✓✓✓
5   | Bronec Ondřej                  | 0.3410     | VYSOKÁ ✓✓✓

--------------------------------------------------------------------------------
1b. Retrieve relevant (s filtrem threshold=1.5):
------------------------------------------------------------


2025-12-17 16:42:39,147 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:39,185 - src.parent_retriever - INFO - Retrieved 5 relevant parent documents (filtered out 0 irrelevant results)
2025-12-17 16:42:39,250 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:39,266 - src.parent_retriever - INFO - Retrieved 5 results with scores
2025-12-17 16:42:39,274 - src.parent_retriever - INFO - Retrieving relevant documents for query: 'React frontend developer with GraphQL and TypeScript' (top 5, threshold 1.5)



Nalezeno 5 relevantních dokumentů:
  1. Baláček Daniel
  2. Huňa Tomáš
  3. Bukovský Petr
  4. Petr Jan
  5. Bronec Ondřej


TEST 2: Detekce relevance - Irelevantní dotaz (React frontend)

Dotaz: "React frontend developer with GraphQL and TypeScript"
(Předpokládáme, že nemáme žádného React vývojáře)

2a. Retrieve s scores (všechny výsledky):
------------------------------------------------------------

#   | Kandidát                       | Score      | Relevance
--------------------------------------------------------------------------------
1   | Bukovský Petr                  | 0.4267     | VYSOKÁ ✓✓✓
2   | Petr Jan                       | 0.4382     | VYSOKÁ ✓✓✓
3   | Němeček Tomáš                  | 0.4433     | VYSOKÁ ✓✓✓
4   | Petr Jan                       | 0.4438     | VYSOKÁ ✓✓✓
5   | Petr Jan                       | 0.4451     | VYSOKÁ ✓✓✓

------------------------------------------------------------
2b. Retrieve relevant (s filtrem threshold=1.5):
------------------------

...


Otázka: Python developer
Počet kontextů: 5
Odpověď:
Based on the provided CV excerpts, the candidates with Python development experience are:

1. **Baláček Daniel** - Intermediate level in SQL and basic knowledge in Python.
2. **Huňa Tomáš** - Basic knowledge in Python.
3. **Bronec Ondřej** - Extensive experience in Python, including packages such as Pandas, NumPy, Matplotlib, Seaborn, TensorFlow, scikit-learn, SciPy, scikit-image, and PySpark. Also experienced in developing AI apps using Streamlit and Bot Framework SDK.
4. **Bukovský Petr** - Experience with Python for data analysis using packages like Pandas, Numpy, Matplotlib, and Seaborn. Also involved in ETL processes and developing Python applications for specific engineering calculations.

Among these, **Bronec Ondřej** has the most extensive and advanced experience in Python development.


4b. Irelevantní dotaz s filtrem:
------------------------------------------------------------


2025-12-17 16:42:52,355 - src.parent_retriever - INFO - Retrieved 4 relevant parent documents (filtered out 0 irrelevant results)
2025-12-17 16:42:52,358 - src.rag_chain - INFO - Retrieved 4 contexts
2025-12-17 16:42:52,366 - src.parent_retriever - INFO - Retrieving documents for query: 'React frontend developer' (top 5)
2025-12-17 16:42:52,474 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:52,506 - src.parent_retriever - INFO - Retrieved 3 parent documents
2025-12-17 16:42:53,120 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:53,125 - src.rag_chain - INFO - Answer generated successfully
2025-12-17 16:42:53,129 - src.rag_chain - INFO - Processing query: 'React frontend developer' (relevance_filter=False)
2025-12-


Otázka: React frontend developer
Počet kontextů: 4
No relevant results: False
Odpověď:
I don't have enough information to answer this question. None of the provided CV excerpts mention experience or skills specifically related to React frontend development.


4c. Irelevantní dotaz BEZ filtru (pro porovnání):
------------------------------------------------------------


2025-12-17 16:42:53,409 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/text-embedding-ada-002-dolphin-1/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:53,443 - src.parent_retriever - INFO - Retrieved 3 parent documents
2025-12-17 16:42:53,807 - httpx - INFO - HTTP Request: POST https://oai-dolphin-1.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-12-17 16:42:53,810 - src.rag_chain - INFO - Answer generated successfully



Otázka: React frontend developer
Počet kontextů: 3
Odpověď:
I don't have enough information to answer this question.


SHRNUTÍ

1. ✓ Config: similarity_threshold = 1.5 (konfigurovatelné)
2. ✓ Nová metoda: retriever.retrieve_relevant() 
   - Filtruje podle threshold
   - Vrací jen relevantní výsledky (může být prázdné!)
3. ✓ Vylepšený RAG chain: 
   - use_relevance_filter=True (default)
   - Detekuje prázdné výsledky
   - Neposílá irelevantní kontext do LLM
4. ✓ Lepší UX: "No candidates found" místo halucinací

TIP: Nastav threshold v config.py podle potřeby:
  - < 1.0  = velmi přísné (jen velmi podobné)
  - 1.0-1.5 = standardní (dobré výsledky)
  - > 1.5  = volné (více výsledků, nižší kvalita)



## Ověření persistence

Zkontrolujte, že data jsou uložena na disku.

In [23]:
import os

print("\n📁 Soubory na disku:")
print("=" * 80)

persist_dir = Path(config.rag.persist_directory)
docstore_dir = persist_dir / "docstore"

print(f"\nVector store directory: {persist_dir}")
if persist_dir.exists():
    print(f"  ✓ Existuje")
    chroma_files = list(persist_dir.glob("*.sqlite3"))
    print(f"  Soubory: {len(chroma_files)} SQLite databází")
else:
    print(f"  ✗ Neexistuje")

print(f"\nDocstore directory: {docstore_dir}")
if docstore_dir.exists():
    print(f"  ✓ Existuje")
    docstore_files = list(docstore_dir.glob("*"))
    print(f"  Soubory: {len(docstore_files)} parent chunks")
else:
    print(f"  ✗ Neexistuje")

print("\n✓ Všechna data jsou uložena a lze je načíst i po restartu")


📁 Soubory na disku:

Vector store directory: chroma_db
  ✓ Existuje
  Soubory: 1 SQLite databází

Docstore directory: chroma_db\docstore
  ✓ Existuje
  Soubory: 61 parent chunks

✓ Všechna data jsou uložena a lze je načíst i po restartu


## Shrnutí

✅ Training dokončen!

**Co bylo vytvořeno:**
- ChromaDB vectorstore s child chunks (pro vyhledávání)
- LocalFileStore docstore s parent chunks (pro kontext)
- Mapování mezi child a parent chunks

**Vše je uloženo na disku:**
- `./chroma_db/` - ChromaDB databáze
- `./chroma_db/docstore/` - Parent chunks

**Další kroky:**
- Otevřete `query.ipynb` pro testování dotazů
- Nebo použijte `train.py` pro automatický training

## Jen vlastní hraní:

In [28]:
print("\n" + "="*80)
print("NÁHLED: Child chunks, které půjdou do vectorstore")
print("="*80)

# Vytvoř splitters pro preview
from langchain_text_splitters import RecursiveCharacterTextSplitter

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.rag.child_chunk_size,
    chunk_overlap=config.rag.child_chunk_overlap,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Ukázka pro první 2 CVčka
print(f"\nUkázka child chunks pro první 2 dokumenty:\n")

for i, doc in enumerate(documents[:5]):
    candidate_name = doc.metadata.get('candidate_name', 'Unknown')
    
    print(f"\n{'='*60}")
    print(f"📄 CV: {candidate_name}")
    print(f"{'='*60}")
    
    # Split na child chunks
    child_chunks = child_splitter.split_documents([doc])
    
    print(f"Počet child chunks: {len(child_chunks)}")
    print(f"\nPrvních 3 child chunks:\n")
    
    for j, chunk in enumerate(child_chunks[:3]):
        print(f"\n--- Child chunk {j+1}/{len(child_chunks)} ---")
        print(f"Velikost: {len(chunk.page_content)} znaků")
        print(f"Metadata: {chunk.metadata}")
        print(f"\nObsah (prvních 300 znaků):")
        print(chunk.page_content[:300] + "...")
        print()

# Celkový přehled
print("\n" + "="*80)
print("CELKOVÝ PŘEHLED - všechny dokumenty")
print("="*80 + "\n")

total_child_chunks = 0
for doc in documents:
    candidate_name = doc.metadata.get('candidate_name', 'Unknown')
    child_chunks = child_splitter.split_documents([doc])
    num_chunks = len(child_chunks)
    total_child_chunks += num_chunks
    print(f"📄 {candidate_name:30s} → {num_chunks:3d} child chunks")

print(f"\n{'='*60}")
print(f"CELKEM: {total_child_chunks} child chunks půjde do vectorstore")
print(f"{'='*60}")


NÁHLED: Child chunks, které půjdou do vectorstore

Ukázka child chunks pro první 2 dokumenty:


📄 CV: Baláček Daniel
Počet child chunks: 10

Prvních 3 child chunks:


--- Child chunk 1/10 ---
Velikost: 388 znaků
Metadata: {'candidate_name': 'Baláček Daniel', 'source': '..\\data\\OneDrive_2025-12-16\\Baláček_Daniel_CV_EN.docx', 'type': 'cv_parent', 'filename': 'Baláček_Daniel_CV_EN.docx', 'file_size': 496940, 'text_length': 3020}

Obsah (prvních 300 znaků):
www.dolphinconsulting.cz	

Daniel Baláček

BI consultant 

 Praha



Key qualifications

Dashboard and report development and design

User requirements analysis and documentation

Business and data analysis



Skills & knowledge

Business intelligence

General Skills

Dashboard and report developmen...


--- Child chunk 2/10 ---
Velikost: 386 znaků
Metadata: {'candidate_name': 'Baláček Daniel', 'source': '..\\data\\OneDrive_2025-12-16\\Baláček_Daniel_CV_EN.docx', 'type': 'cv_parent', 'filename': 'Baláček_Daniel_CV_EN.docx', 'file_si

In [29]:
print("\n" + "="*80)
print("NÁHLED: Parent + Child documents vztahy")
print("="*80)

from langchain_text_splitters import RecursiveCharacterTextSplitter

# Vytvoř oba splitters
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.rag.parent_chunk_size,
    chunk_overlap=config.rag.parent_chunk_overlap,
    separators=["\n\n", "\n", ". ", " ", ""]
)

child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.rag.child_chunk_size,
    chunk_overlap=config.rag.child_chunk_overlap,
    separators=["\n\n", "\n", ". ", " ", ""]
)

# Ukázka pro první 2 CVčka
print(f"\nUkázka parent → child vztahů pro první 2 dokumenty:\n")

total_parents = 0
total_children = 0

for i, doc in enumerate(documents[:2]):
    candidate_name = doc.metadata.get('candidate_name', 'Unknown')
    
    print(f"\n{'='*70}")
    print(f"📄 CV: {candidate_name}")
    print(f"   Původní velikost: {len(doc.page_content)} znaků")
    print(f"{'='*70}")
    
    # 1. Split na parent chunks
    parent_chunks = parent_splitter.split_documents([doc])
    num_parents = len(parent_chunks)
    total_parents += num_parents
    
    print(f"\n→ Rozděleno na {num_parents} parent chunks:")
    
    # 2. Pro každý parent chunk ukázat child chunks
    doc_child_count = 0
    for p_idx, parent in enumerate(parent_chunks, 1):
        parent_id = f"parent-{i}-{p_idx}"  # Simulace ID
        
        # Split parent na child chunks
        child_chunks = child_splitter.split_documents([parent])
        num_children = len(child_chunks)
        doc_child_count += num_children
        total_children += num_children
        
        print(f"\n  📦 Parent chunk {p_idx}/{num_parents}")
        print(f"     Velikost: {len(parent.page_content)} znaků")
        print(f"     ID: {parent_id}")
        print(f"     → Obsahuje {num_children} child chunks:")
        
        for c_idx, child in enumerate(child_chunks, 1):
            print(f"        └─ Child {c_idx}: {len(child.page_content)} znaků (parent_id: {parent_id})")
        
        # Ukázka obsahu prvního parent chunku
        if p_idx == 1:
            print(f"\n     Obsah parent chunku (prvních 200 znaků):")
            print(f"     {parent.page_content[:200]}...")
            
            print(f"\n     První child chunk z tohoto parent:")
            print(f"     {child_chunks[0].page_content[:150]}...")
    
    print(f"\n  CELKEM pro {candidate_name}:")
    print(f"    - {num_parents} parent chunks")
    print(f"    - {doc_child_count} child chunks")

# Celkový přehled pro všechny dokumenty
print("\n" + "="*80)
print("CELKOVÝ PŘEHLED - všechny dokumenty")
print("="*80 + "\n")

print(f"{'CV jméno':<30} | {'Parents':<10} | {'Children':<10} | Children/Parent")
print("-" * 80)

grand_total_parents = 0
grand_total_children = 0

for doc in documents:
    candidate_name = doc.metadata.get('candidate_name', 'Unknown')
    
    # Split na parents
    parent_chunks = parent_splitter.split_documents([doc])
    num_parents = len(parent_chunks)
    
    # Spočítej children
    num_children = 0
    for parent in parent_chunks:
        child_chunks = child_splitter.split_documents([parent])
        num_children += len(child_chunks)
    
    grand_total_parents += num_parents
    grand_total_children += num_children
    
    ratio = num_children / num_parents if num_parents > 0 else 0
    
    print(f"{candidate_name:<30} | {num_parents:<10} | {num_children:<10} | {ratio:.1f}")

print("-" * 80)
print(f"{'CELKEM':<30} | {grand_total_parents:<10} | {grand_total_children:<10} | {grand_total_children/grand_total_parents:.1f}")

print(f"\n{'='*80}")
print(f"CO SE ULOŽÍ:")
print(f"  - Vectorstore (ChromaDB): {grand_total_children} child chunks (s embeddingy)")
print(f"  - Docstore (disk):        {grand_total_parents} parent chunks (jako JSON)")
print(f"{'='*80}")


NÁHLED: Parent + Child documents vztahy

Ukázka parent → child vztahů pro první 2 dokumenty:


📄 CV: Baláček Daniel
   Původní velikost: 3020 znaků

→ Rozděleno na 2 parent chunks:

  📦 Parent chunk 1/2
     Velikost: 1995 znaků
     ID: parent-0-1
     → Obsahuje 7 child chunks:
        └─ Child 1: 388 znaků (parent_id: parent-0-1)
        └─ Child 2: 386 znaků (parent_id: parent-0-1)
        └─ Child 3: 224 znaků (parent_id: parent-0-1)
        └─ Child 4: 260 znaků (parent_id: parent-0-1)
        └─ Child 5: 241 znaků (parent_id: parent-0-1)
        └─ Child 6: 391 znaků (parent_id: parent-0-1)
        └─ Child 7: 241 znaků (parent_id: parent-0-1)

     Obsah parent chunku (prvních 200 znaků):
     www.dolphinconsulting.cz	

Daniel Baláček

BI consultant 

 Praha



Key qualifications

Dashboard and report development and design

User requirements analysis and documentation

Business and data an...

     První child chunk z tohoto parent:
     www.dolphinconsulting.cz	

Daniel Balá